In [ ]:
import minari
import os
import h5py
import numpy as np
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
import shimmy
from custom_dmc_tasks import point_mass_maze

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise

from torch.utils.data import Dataset, DataLoader
# os.environ['MUJOCO_GL'] = 'glfw'

In [2]:
task = 'reach_top_right'



# TD3 Training

In [ ]:
def render_environment():
    raw_env = point_mass_maze.make(task=task)
    raw_env.reset()
    
    physics = raw_env.physics
    frame = physics.render(
            height=480, 
            width=480, 
            camera_id=0, 
            )

    plt.figure(figsize=(8, 8))
    plt.imshow(frame)
    plt.axis('off')
    plt.title("PointMass Maze")
    plt.show()


In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

def make_env():
    raw_env = point_mass_maze.make(task=task)
    return shimmy.DmControlCompatibilityV0(raw_env)

env = DummyVecEnv([make_env for _ in range(4)])
env = VecMonitor(env)


policy_kwargs = dict(
    net_arch=dict(
        pi=[512, 512, 512],  # (Actor/Policy)
        qf=[512, 512, 512]   # (Critic/Q-function)
    )
)

model = TD3(
    "MultiInputPolicy",
    env,
    policy_kwargs=policy_kwargs,
    learning_rate=1e-4,      
    buffer_size=1_000_000,
    batch_size=256,        
    tau=0.005,                
    gamma=0.99,              
    target_policy_noise=0.2, 
    target_noise_clip=0.3,
    policy_delay=2,          
    verbose=1,
    device='cpu'
)


In [ ]:
model.learn(total_timesteps=5000_000, progress_bar=True)

In [ ]:
model.save(f"td3_point_mass_expert_{task}")

In [ ]:
import imageio
import numpy as np

def create_agent_gif(model_path="td3_point_mass_expert", gif_name="agent_trajectory.gif"):
    raw_env = point_mass_maze.make(task=task)
    env = shimmy.DmControlCompatibilityV0(raw_env) 
    
    model = TD3.load(model_path)
    
    obs, _ = env.reset()
    frames = []
    

    for _ in range(500): 
        frame = raw_env.physics.render(height=480, width=480, camera_id=0)
        frames.append(frame)
        
        action, _ = model.predict(obs, deterministic=True)
        obs, _, terminated, truncated, _ = env.step(action)
        
        if terminated or truncated:
            break

    if frames:
        imageio.mimsave(gif_name, [np.array(f).astype(np.uint8) for f in frames], 
                        fps=30) 
    else:
        print("Ошибка: кадры не были сгенерированы!")

if __name__ == "__main__":
    create_agent_gif(model_path=f"td3_point_mass_expert_{task}", 
                     gif_name=f"agent_trajectory_{task}.gif")